# Fake News Detector

## Import Library

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

## Import Data

In [3]:
true = pd.read_csv('True.csv', engine='python', encoding='utf-8', on_bad_lines='skip')
fake = pd.read_csv('Fake.csv', engine='python', encoding='utf-8', on_bad_lines='skip')

In [4]:
true['label'] = 1
fake['label'] = 0

# Data Preprocessing

## Data Integration

In [5]:
news = pd.concat([fake, true], axis=0)

In [6]:
news.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [7]:
news.tail()

,title,text,subject,date,label
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


## Data Cleaning

### Checking Null Values

In [8]:
news.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

### Dropping Unnecessary Column

In [9]:
news.drop(['title', 'subject', 'date'], axis=1)

,text,label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
21412,BRUSSELS (Reuters) - NATO allies on Tuesday we...,1
21413,"LONDON (Reuters) - LexisNexis, a provider of l...",1
21414,MINSK (Reuters) - In the shadow of disused Sov...,1
21415,MOSCOW (Reuters) - Vatican Secretary of State ...,1


In [10]:
news

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


### Scramble Data

In [11]:
news = news.sample(frac=1)
news.reset_index(inplace=True)
news.drop(['index'], axis = 1, inplace=True)

In [12]:
news

,title,text,subject,date,label
0,UNHOLY ALLIANCE: Hillary Clinton’s Saudi Spons...,21st Century Wire says Amid the tossing and t...,US_News,"August 26, 2016",0
1,ALL HELL IS ABOUT TO BREAK LOOSE Between Europ...,European nations have naively opened their bor...,left-news,"Feb 25, 2016",0
2,Senate blocks Democrats' plan to deny firearms...,WASHINGTON (Reuters) - The U.S. Senate on Mond...,politicsNews,"June 20, 2016",1
3,U.S. vetoes U.N. call for withdrawal of Trump ...,UNITED NATIONS (Reuters) - The United States w...,worldnews,"December 18, 2017",1
4,"Obama Exposes Trump For The World To See, And...","President Obama, due to his unique position, i...",News,"June 29, 2016",0
...,...,...,...,...,...
44893,U.S. intel committee subpoenas comedian in Rus...,(Reuters) - A New York comedian has been compe...,worldnews,"November 30, 2017",1
44894,Ladies Of ‘The View’ Hilariously ROAST Trump ...,With Donald Trump throwing yet another hissy f...,News,"December 5, 2016",0
44895,HERE’S THIS LIST OF REPUBLICANS Running For Re...,Here is the list of Republicans who won t supp...,left-news,"Oct 20, 2016",0
44896,U.S. has 'ironclad' commitment to NATO mutual ...,WASHINGTON (Reuters) - The United States has a...,politicsNews,"July 21, 2016",1


## Feature Extraction (TF-IDF)

In [13]:
import re

In [14]:
def wordopt(text):
    # Convert into lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'https?://\S+|\www\.\S+', '', text)

    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove digits
    text = re.sub(r'\d', '', text)

    # Remove newline characters
    text = re.sub(r'\n', ' ', text)

    return text


In [15]:
news['text'] = news['text'].apply(wordopt)

In [16]:
news['text']

0         st century wire says amid the tossing and tur...
1        european nations have naively opened their bor...
2        washington reuters  the us senate on monday bl...
3        united nations reuters  the united states was ...
4        president obama due to his unique position is ...
                               ...                        
44893    reuters  a new york comedian has been compelle...
44894    with donald trump throwing yet another hissy f...
44895    here is the list of republicans who won t supp...
44896    washington reuters  the united states has an i...
44897    yada yada yada  hillary clinton announced her ...
Name: text, Length: 44898, dtype: object

## Stopwords and Stemming

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

In [19]:
def preprocess_text(text):
    # Tokenisasi teks
    words = text.split()

    # Hapus stopwords dan stem kata
    processed_words = [stemmer.stem(word) for word in words if word.lower() not in stop_words]

    # Gabungkan kembali kata yang telah diproses
    return ' '.join(processed_words)

## Data Transformation

In [20]:
x = news['text']
y = news['label']

In [21]:
x.shape

(44898,)

### TF-IDF Vectorizer

In [22]:
vectorization = TfidfVectorizer()

In [23]:
x_tfidf = vectorization.fit_transform(x)

### Stemming and Stopwords

In [24]:
X_train_processed = [preprocess_text(sentence) for sentence in x]

In [25]:
def custom_tokenizer(text):
    return preprocess_text(text).split()

In [26]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer)

In [27]:
x_vectorized = vectorizer.fit_transform(X_train_processed)

c:\Users\Kalea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


# Split Data

### Split Data: TF-IDF

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x_tfidf, y, test_size=0.3, random_state=10)

Split Data: Stemming & Stopwords

In [29]:
x_train_stem, x_test_stem, y_train_stem, y_test_stem = train_test_split(x_vectorized, y, test_size=0.3, random_state=10)

# Model

## Pembuatan Model Logistic Regression

### Model Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
tf_idf_LR = LogisticRegression()
stem_LR = LogisticRegression()

In [32]:
tf_idf_LR.fit(x_train, y_train)
stem_LR.fit(x_train_stem, y_train_stem)

c:\Users\Kalea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
tf_idf_train_pred_lr = tf_idf_LR.predict(x_train)
tf_idf_test_pred_lr = tf_idf_LR.predict(x_test)

stem_train_pred_lr = stem_LR.predict(x_train_stem)
stem_test_pred_lr = stem_LR.predict(x_test_stem)

In [34]:
tf_idf_LR.score(x_train, y_train)

0.9926816851215476

In [35]:
tf_idf_LR.score(x_test, y_test)

0.9889383815887156

In [36]:
stem_LR.score(x_train_stem, y_train_stem)

0.9999681812396589

In [37]:
stem_LR.score(x_test_stem, y_test_stem)

0.9967334818114328

In [38]:
print(classification_report(y_train, tf_idf_train_pred_lr))
print(classification_report(y_test, tf_idf_test_pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     16473
           1       0.99      0.99      0.99     14955

    accuracy                           0.99     31428
   macro avg       0.99      0.99      0.99     31428
weighted avg       0.99      0.99      0.99     31428

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7008
           1       0.99      0.99      0.99      6462

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [39]:
print(classification_report(y_train_stem, stem_train_pred_lr))
print(classification_report(y_test_stem, stem_test_pred_lr))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16473
           1       1.00      1.00      1.00     14955

    accuracy                           1.00     31428
   macro avg       1.00      1.00      1.00     31428
weighted avg       1.00      1.00      1.00     31428

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7008
           1       1.00      1.00      1.00      6462

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



### Optimasi Parameter Pada Logistic Regression

In [40]:
# Definisikan parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

In [41]:
from sklearn.model_selection import GridSearchCV

# Definisikan parameter grid
tf_idf_param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Definisikan GridSearchCV
tf_idf_grid_search_log_reg = GridSearchCV(estimator=stem_LR, param_grid=tf_idf_param_grid, cv=5, scoring='accuracy')

# Fit model
tf_idf_grid_search_log_reg.fit(x_train, y_train)


# Definisikan parameter grid
stem_param_grid = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# Definisikan GridSearchCV
stem_grid_search_log_reg = GridSearchCV(estimator=stem_LR, param_grid=stem_param_grid, cv=5, scoring='accuracy')

# Fit model
stem_grid_search_log_reg.fit(x_train_stem, y_train_stem)



c:\Users\Kalea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Kalea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\Kalea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1183: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
c:\Users\Kalea\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\Kalea\AppData\Local\Programs\Python\Python310\lib\s

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2', 'elasticnet', 'none'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [42]:
# Hasil terbaik
print(f"Best parameters for TF IDF Logistic Regression: {tf_idf_grid_search_log_reg.best_params_}")
print(f"Best parameters for Stemming and Stopwords Logistic Regression: {stem_grid_search_log_reg.best_params_}")

Best parameters for TF IDF Logistic Regression: {'C': 100, 'penalty': 'l1', 'solver': 'liblinear'}
Best parameters for Stemming and Stopwords Logistic Regression: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}


### Model Setelah Optimisasi

## Pembuatan Model Decision Tree Classifier

### Model Decision Tree Classifier

In [43]:
from sklearn.tree import DecisionTreeClassifier

In [44]:
tf_idf_DTC = DecisionTreeClassifier()
stem_DTC = DecisionTreeClassifier()

In [45]:
# DTC.fit(x_train, y_train)

tf_idf_DTC.fit(x_train, y_train)
stem_DTC.fit(x_train_stem, y_train_stem)

DecisionTreeClassifier()

In [46]:
# train_pred_dtc = DTC.predict(x_train)
# test_pred_dtc = DTC.predict(x_test)

tf_idf_train_pred_DTC = tf_idf_DTC.predict(x_train)
tf_idf_test_pred_DTC = tf_idf_DTC.predict(x_test)

stem_train_pred_DTC = stem_DTC.predict(x_train_stem)
stem_test_pred_DTC = stem_DTC.predict(x_test_stem)

In [47]:
tf_idf_DTC.score(x_train, y_train)


1.0

In [48]:
tf_idf_DTC.score(x_test, y_test)

0.9957683741648107

In [49]:
stem_DTC.score(x_train_stem, y_train_stem)

1.0

In [50]:
stem_DTC.score(x_test_stem, y_test_stem)

0.9959910913140312

In [51]:
print(classification_report(y_train, tf_idf_train_pred_DTC))
print(classification_report(y_test, tf_idf_test_pred_DTC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16473
           1       1.00      1.00      1.00     14955

    accuracy                           1.00     31428
   macro avg       1.00      1.00      1.00     31428
weighted avg       1.00      1.00      1.00     31428

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7008
           1       1.00      1.00      1.00      6462

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



In [52]:
print(classification_report(y_train_stem, stem_train_pred_DTC))
print(classification_report(y_test_stem, stem_test_pred_DTC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16473
           1       1.00      1.00      1.00     14955

    accuracy                           1.00     31428
   macro avg       1.00      1.00      1.00     31428
weighted avg       1.00      1.00      1.00     31428

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7008
           1       1.00      1.00      1.00      6462

    accuracy                           1.00     13470
   macro avg       1.00      1.00      1.00     13470
weighted avg       1.00      1.00      1.00     13470



### Optimasi Parameter Pada Decision Tree classifier

In [53]:
from sklearn.model_selection import GridSearchCV

# Definisikan parameter grid
tf_idf_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Definisikan GridSearchCV
tf_idf_grid_search_dt = GridSearchCV(estimator=tf_idf_DTC, param_grid=tf_idf_param_grid, cv=5, scoring='accuracy')

# Fit model
tf_idf_grid_search_dt.fit(x_train, y_train)

stem_param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Definisikan GridSearchCV
stem_grid_search_dt = GridSearchCV(estimator=stem_DTC, param_grid=stem_param_grid, cv=5, scoring='accuracy')

# Fit model
stem_grid_search_dt.fit(x_train_stem, y_train_stem)



GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='accuracy')

In [54]:

print(f"Best parameters for TF IDF Decision Tree Classifier: {tf_idf_grid_search_dt.best_params_}")

print(f"Best parameters for Stem and Stopwords Decision Tree Classifier: {stem_grid_search_dt.best_params_}")

Best parameters for TF IDF Decision Tree Classifier: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 10}
Best parameters for Stem and Stopwords Decision Tree Classifier: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10}


### Model DTC Setelah Optimisasi

## Pembuatan Model Random Forest Classifier

### Model Random Forest Classifier

In [55]:
from sklearn.ensemble import RandomForestClassifier

In [56]:
tf_idf_RFC = RandomForestClassifier()
stem_RFC = RandomForestClassifier()

In [57]:
tf_idf_RFC.fit(x_train, y_train)
stem_RFC.fit(x_train_stem, y_train_stem)

RandomForestClassifier()

In [58]:
tf_idf_train_pred_RFC = tf_idf_RFC.predict(x_train)
tf_idf_test_pred_RFC = tf_idf_RFC.predict(x_test)

stem_train_pred_RFC = stem_RFC.predict(x_train_stem)
stem_test_pred_RFC = stem_RFC.predict(x_test_stem)

In [59]:
tf_idf_RFC.score(x_train, y_train)


1.0

In [60]:
tf_idf_RFC.score(x_test, y_test)


0.9878247958426132

In [61]:
stem_RFC.score(x_train_stem, y_train_stem)


1.0

In [62]:
stem_RFC.score(x_test_stem, y_test_stem)

0.984558277654046

In [63]:
print(classification_report(y_train, tf_idf_train_pred_RFC))
print(classification_report(y_test, tf_idf_test_pred_RFC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16473
           1       1.00      1.00      1.00     14955

    accuracy                           1.00     31428
   macro avg       1.00      1.00      1.00     31428
weighted avg       1.00      1.00      1.00     31428

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7008
           1       0.99      0.99      0.99      6462

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [64]:
print(classification_report(y_train_stem, stem_train_pred_RFC))
print(classification_report(y_test_stem, stem_test_pred_RFC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     16473
           1       1.00      1.00      1.00     14955

    accuracy                           1.00     31428
   macro avg       1.00      1.00      1.00     31428
weighted avg       1.00      1.00      1.00     31428

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      7008
           1       0.98      0.99      0.98      6462

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



### Optimasi Parameter Pada Random Forest Classifier

In [65]:
from sklearn.model_selection import GridSearchCV


# Definisikan parameter grid
tf_idf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Definisikan GridSearchCV
tf_idf_grid_search_rf = GridSearchCV(estimator=tf_idf_RFC, param_grid=tf_idf_param_grid, cv=5, scoring='accuracy')

# Fit model
tf_idf_grid_search_rf.fit(x_train, y_train)

# Hasil terbaik
print(f"Best parameters for Random Forest Classifier: {tf_idf_grid_search_rf.best_params_}")

# Definisikan parameter grid
stem_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Definisikan GridSearchCV
stem_grid_search_rf = GridSearchCV(estimator=stem_RFC, param_grid=param_grid, cv=5, scoring='accuracy')

# Fit model
stem_grid_search_rf.fit(x_train_stem, y_train_stem)

# Hasil terbaik
print(f"Best parameters for Random Forest Classifier: {stem_grid_search_rf.best_params_}")


Best parameters for Random Forest Classifier: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


ValueError: Invalid parameter 'C' for estimator RandomForestClassifier(). Valid parameters are: ['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'].

Best parameters for TF IDF Random Forest Classifier: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}

In [66]:

print(f"Best parameters for TF IDF Random Forest Classifier: {tf_idf_grid_search_rf.best_params_}")
print(f"Best parameters for Stem and Stopwords Random Forest Classifier: {stem_grid_search_rf.best_params_}")

Best parameters for TF IDF Random Forest Classifier: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

### Model Setelah Optimisasi

# TESTING

In [ ]:
def output_label(n):
  if n==0:
    return "It Is Fake News"
  elif n==1:
    return "It Is Genuine News"


In [ ]:
def manual_testing(news):
    # Corrected syntax for defining dictionary
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_x_test = new_def_test["text"].apply(wordopt)

    # Assuming 'vectorization' is your vectorizer object
    new_xv_test = vectorization.transform(new_x_test)

    # Model predictions
    pred_lr = LR.predict(new_xv_test)
    pred_dtc = DTC.predict(new_xv_test)
    pred_rfc = RFC.predict(new_xv_test)

    return "\nLR Prediction: {}\nDTC Prediction: {}\nRFC Prediction: {}".format(
        output_label(pred_lr[0]),
        output_label(pred_dtc[0]),
        output_label(pred_rfc[0])
    )


In [ ]:
news_article = str(input())

KeyboardInterrupt: Interrupted by user

In [ ]:
manual_testing(news_article)